In [2]:
def kmeans(X, k, max_iters=100, tol=1e-4):
    """
    自实现K-means算法。
    
    :param X: 输入数据集，形状为 (样本数, 特征数)，表示包含多个样本和其对应特征的二维数组。
    :param k: 聚类的簇数，即我们希望将数据分为多少簇类。
    :param max_iters: 最大迭代次数，用于控制算法的执行上限。
    :param tol: 收敛阈值，表示簇中心在两次更新之间的移动距离小于该值时停止迭代，相当于个早停。
    :return: 一个元组 (labels, centers)，其中：
             - labels: 聚类标签数组，形状为 (样本数,)，表示每个样本所属的簇。
             - centers: 簇中心数组，形状为 (k, 特征数)，表示每个簇的中心位置。
    """
    np.random.seed(42)  # 设置随机种子，保证结果可复现。
    
    # 随机初始化簇中心，从数据集中随机选取 k 个样本作为初始中心。
    centers = X[np.random.choice(X.shape[0], k, replace=False)]
    
    # 初始化每个样本的标签，labels 用于记录每个样本当前所属的簇。
    labels = np.zeros(X.shape[0])
    
    # 开始迭代更新簇中心
    for iteration in range(max_iters):
        # Step 1: 分配样本到最近的簇中心
        for i, sample in enumerate(X):  # 遍历每个样本
            # 计算当前样本与每个簇中心的欧几里得距离
            distances = np.linalg.norm(sample - centers, axis=1)
            # 将样本分配给距离最近的簇中心
            labels[i] = np.argmin(distances)
        
        # Step 2: 计算新的簇中心
        # 根据分配结果，计算每个簇的平均位置作为新的簇中心
        new_centers = np.array([X[labels == j].mean(axis=0) for j in range(k)])
        
        # Step 3: 检查是否收敛
        # 如果所有簇中心的移动距离小于收敛阈值 tol，则停止迭代
        if np.linalg.norm(new_centers - centers) < tol:
            break  # 收敛条件满足，退出循环
        
        # 更新簇中心为新计算的中心
        centers = new_centers
    
    # 返回最终的聚类标签和簇中心
    return labels, centers


In [3]:
def kmeans_pp_init(X, k):
    """
    使用 K-means++ 方法初始化簇中心。
    
    :param X: 数据集，形状为 (样本数, 特征数)，包含多个样本和其对应的特征。
    :param k: 簇的数量，即希望生成的初始中心数目。
    :return: 初始化的簇中心，形状为 (k, 特征数)。
    """
    np.random.seed(42)  # 设置随机种子，保证结果可复现
    n_samples = X.shape[0]  # 数据集中的样本数量
    centers = []  # 用于存储初始化的簇中心
    
    # Step 1: 随机选择第一个中心
    # 从数据集中随机选择一个样本作为第一个簇中心
    first_center = X[np.random.choice(n_samples)]
    centers.append(first_center)
    
    # Step 2: 逐步选择剩余的 k-1 个中心
    for _ in range(k - 1):
        # 计算每个样本到最近一个已选择中心的平方距离
        # 对于每个中心，计算所有样本到它的距离，然后取这些距离的最小值
        distances = np.min([np.linalg.norm(X - center, axis=1)**2 for center in centers], axis=0)
        
        # 根据距离分布计算每个样本作为中心的概率
        probabilities = distances / distances.sum()
        
        # 按概率分布随机选择下一个簇中心
        next_center = X[np.random.choice(n_samples, p=probabilities)]
        centers.append(next_center)
    
    return np.array(centers)  # 返回初始化的簇中心数组

In [4]:
def kmeans_with_pp(X, k, max_iters=100, tol=1e-4):
    """
    自实现 K-means 聚类算法，使用 K-means++ 方法初始化簇中心。
    
    :param X: 数据集，形状为 (样本数, 特征数)。
    :param k: 簇的数量。
    :param max_iters: 最大迭代次数，用于控制算法的执行上限。
    :param tol: 收敛阈值，当簇中心更新的移动距离小于此值时停止迭代。
    :return: 聚类标签和簇中心，其中：
             - labels: 聚类标签数组，形状为 (样本数,)。
             - centers: 最终的簇中心，形状为 (k, 特征数)。
    """
    # Step 1: 使用 K-means++ 初始化簇中心
    centers = kmeans_pp_init(X, k)
    
    # 初始化每个样本的标签，labels 用于记录样本所属的簇
    labels = np.zeros(X.shape[0])
    
    # Step 2: 开始迭代更新簇中心
    for iteration in range(max_iters):
        # Step 2.1: 将样本分配到最近的簇中心
        for i, sample in enumerate(X):  # 遍历每个样本
            # 计算样本到所有簇中心的欧几里得距离
            distances = np.linalg.norm(sample - centers, axis=1)
            # 将样本分配给距离最近的簇
            labels[i] = np.argmin(distances)
        
        # Step 2.2: 计算新的簇中心
        # 按照样本分配结果，计算每个簇的平均位置作为新的簇中心
        new_centers = np.array([X[labels == j].mean(axis=0) for j in range(k)])
        
        # Step 2.3: 检查是否收敛
        # 如果所有簇中心的移动距离小于收敛阈值 tol，则停止迭代
        if np.linalg.norm(new_centers - centers) < tol:
            break  # 收敛条件满足，退出循环
        
        # 更新簇中心为新计算的值
        centers = new_centers
    
    # 返回最终的聚类标签和簇中心
    return labels, centers

In [5]:
import numpy as np
from sklearn.datasets import make_blobs
from sklearn.metrics import silhouette_score

# 生成示例数据集
np.random.seed(42)
X, _ = make_blobs(n_samples=500, centers=5, cluster_std=0.6, random_state=42)

# 多次运行比较性能
k = 5
n_runs = 10
results_kmeans = []
results_kmeans_pp = []

for _ in range(n_runs):
    # K-means
    labels, centers = kmeans(X, k)
    sse = np.sum([np.linalg.norm(X[labels == j] - center, axis=1).sum() for j, center in enumerate(centers)])
    silhouette = silhouette_score(X, labels)
    results_kmeans.append((sse, silhouette))
    
    # K-means++
    labels_pp, centers_pp = kmeans_with_pp(X, k)
    sse_pp = np.sum([np.linalg.norm(X[labels_pp == j] - center, axis=1).sum() for j, center in enumerate(centers_pp)])
    silhouette_pp = silhouette_score(X, labels_pp)
    results_kmeans_pp.append((sse_pp, silhouette_pp))

# 结果汇总
sse_kmeans, silhouette_kmeans = zip(*results_kmeans)
sse_kmeans_pp, silhouette_kmeans_pp = zip(*results_kmeans_pp)

print(f"K-means SSE: Mean = {np.mean(sse_kmeans):.2f}, Std = {np.std(sse_kmeans):.2f}")
print(f"K-means Silhouette Score: Mean = {np.mean(silhouette_kmeans):.4f}, Std = {np.std(silhouette_kmeans):.4f}")

print(f"K-means++ SSE: Mean = {np.mean(sse_kmeans_pp):.2f}, Std = {np.std(sse_kmeans_pp):.2f}")
print(f"K-means++ Silhouette Score: Mean = {np.mean(silhouette_kmeans_pp):.4f}, Std = {np.std(silhouette_kmeans_pp):.4f}")


K-means SSE: Mean = 568.53, Std = 0.00
K-means Silhouette Score: Mean = 0.7091, Std = 0.0000
K-means++ SSE: Mean = 366.21, Std = 0.00
K-means++ Silhouette Score: Mean = 0.7993, Std = 0.0000
